# PDMS 3D model contextualization
This notebook contains a workflow for contextualizing 3D models made in PDMS which typically works well for oil and gas 3D models. 

Authors: Alina Astrakova and Anders Hafreager

In [1]:
import pandas as pd

from getpass import getpass
from cognite.experimental import CogniteClient
from cognite.client.data_classes.three_d import ThreeDAssetMapping

# Initialize

In [2]:
client = CogniteClient(
    api_key=getpass(), 
    project="publicdata", 
    client_name="dshub"
)
client.login.status()

········


/Users/sindrest/projects/pdf-pnid-parsing/.venv/lib/python3.7/site-packages/cognite/client/_cognite_client.py:66: UserWarning: You are using version 1.7.0 of the SDK, however version 1.8.1 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  debug=debug,


{
    "user": "sindre.stavseng@cognite.com",
    "project": "publicdata",
    "project_id": 5977964818434649,
    "logged_in": true,
    "api_key_id": 3612386474258642
}

In [3]:
# Find the 3D model you want to contextualize
client.three_d.models.list(limit=-1)

,name,id,created_time,metadata
0,Valhall PH,4715379429968321,1587019342625,{}


In [5]:
# For a given model (and its model id), find the 3D model revision you want to contextualize
# If you don't find the revision, try published=False
client.three_d.revisions.list(model_id=4715379429968321, published=True) 

,id,file_id,published,camera,status,metadata,thumbnail_threed_file_id,thumbnail_url,asset_mapping_count,created_time
0,5688854005909501,1339722896977352,True,"{'target': [135.94219970703125, 113.1478958129...",Done,{},7591221097548162,https://api.cognitedata.com/api/v1/projects/pu...,92,1587019381210


In [6]:
# List all root assets to find the root asset you want to map to
client.assets.list(root=True, limit=-1)

,external_id,name,metadata,id,created_time,last_updated_time,root_id,description
0,houston.00. Support systems.Reverse osmosis,Reverse osmosis,"{'_replicatedInternalId': '1536954437306151', ...",5072327905985771,1592572207249,1592578295754,5072327905985771,NaN
1,NaN,Aker BP,{},6687602007296940,0,0,6687602007296940,Aker BP


In [7]:
# Define 3d model_id and revision
model_id = 4715379429968321
revision_id = 5688854005909501

# Define root_id for assets
root_id = 6687602007296940

# Download data

In [8]:
# Download 3D nodes. This may take a while ...
threed_nodes = client.three_d.revisions.list_nodes(model_id=model_id, revision_id=revision_id, limit=-1)

In [12]:
# The 3D node hierarchy is often made of nodes with names on the form "/21PT1019"
# with children nodes with names "BRANCH 1 of /21PT1019".
# We only want to map the parent node, so remove all nodes with such names.
nodes_list = threed_nodes.dump()
filtered_nodes = list(filter(lambda x: x["name"] != "", nodes_list))
print("%d non empty node names" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "EQUIPMENT" not in x["name"], filtered_nodes))
print("%d node names without EQUIPMENT" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "BRANCH" not in x["name"], filtered_nodes))
print("%d node names without BRANCH" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "STRUCTURE" not in x["name"], filtered_nodes))
print("%d node names without STRUCTURE" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: " OF " not in x["name"], filtered_nodes))
print("%d node names without OF " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: " of " not in x["name"], filtered_nodes))
print("%d node names without of " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Box" not in x["name"], filtered_nodes))
print("%d node names without Box " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Cylinder" not in x["name"], filtered_nodes))
print("%d node names without Cylinder " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Facet Group" not in x["name"], filtered_nodes))
print("%d node names without Facet Group " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "curve" not in x["name"], filtered_nodes))
print("%d node names without curve " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Pyramid" not in x["name"], filtered_nodes))
print("%d node names without Pyramid " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Line" not in x["name"], filtered_nodes))
print("%d node names without Line " % len(filtered_nodes))
filtered_node_names = list(map(lambda x: {"name": x["name"], "id": x["id"]}, filtered_nodes))

739350 non empty node names
651467 node names without EQUIPMENT
547636 node names without BRANCH
428095 node names without STRUCTURE
428095 node names without OF 
162679 node names without of 
162679 node names without Box 
162678 node names without Cylinder 
162678 node names without Facet Group 
162678 node names without curve 
162678 node names without Pyramid 
162678 node names without Line 


In [13]:
# Download assets
assets = client.assets.retrieve_subtree(root_id)
asset_names_ids = [{"name": x.name, "id": x.id} for x in assets]
print(f"Downloaded {len(asset_names_ids)} assets")

Downloaded 1106 assets


# Run the entity matcher

In [19]:
# Create an entity matching model
model = client.entity_matching.fit_ml(match_from=filtered_node_names, match_to=asset_names_ids)

In [20]:
# Get predictions from model
job = model.predict_ml()

In [24]:
# Check predictions job status. The job will take a few minutes to complete
print(f"Job is {job.update_status()}, last updated at {job.status_timestamp}")

Job is Completed, last updated at 1597649956544


In [25]:
# When the job is finished, get the results
matches = job.result["items"]

In [28]:
# This may require some work of verification. Set a threshold and run 
threshold = 0.9
good_matches = [m for m in matches if (len(m["matches"]) > 0 and m["matches"][0]["score"] > threshold)]
print("Got %d matches with score > %f" % (len(good_matches), threshold))
pd.DataFrame.from_records([(m["matchFrom"]["name"], m["matches"][0]["matchTo"]["name"], m["matches"][0]["score"]) for m in good_matches])

Got 140 matches with score > 0.900000


,0,1,2
0,/COPY_OF_TIE_IN_23,23,1.0
1,/OE150-WELDING-BRACKET-23,23,1.0
2,/23-GK-9107A-A01,23-GK-9107A-A01,1.0
3,/23-GK-9107A-A01/CS,23-GK-9107A-A01,1.0
4,/23-GK-9107A-A01/SUPP,23-GK-9107A-A01,1.0
...,...,...,...
135,/JP769.Structure.Profil_23,23,1.0
136,/Copy-of-Product__23,23,1.0
137,TMP-T23-SCAN_MM.nwd,23,1.0
138,/T23-SCAN_MM,23,1.0


# Create asset mappings

In [31]:
asset_mappings = []
for match in good_matches:
    asset_id = match["matches"][0]["matchTo"]["id"]
    node_id = match["matchFrom"]["id"]
    asset_mappings.append(ThreeDAssetMapping(node_id=node_id, asset_id=asset_id))

In [ ]:
# Write mappings to CDF
res = client.three_d.asset_mappings.create(model_id=model_id, revision_id=revision_id, asset_mapping=asset_mappings)

In [ ]:
# Delete all existing mappings (if you want to redo it)
res = client.three_d.asset_mappings.list(model_id=model_id, revision_id=revision_id, limit=-1)
existing_mappings = list(map(lambda x: ThreeDAssetMapping(node_id=x.node_id, asset_id=x.asset_id), res))
res = client.three_d.asset_mappings.delete(model_id=model_id, revision_id=revision_id, asset_mapping=existing_mappings)